In [1]:
# ! pip install imutils 

In [2]:
from imutils.video import VideoStream
import argparse
import imutils                  #video transitions, rotations
import time
import cv2

In [14]:
args = {'video_name':"football video.mp4",'tracker':'kcf'} 

In [15]:
opencv_ob_track = { 'csrt': cv2.TrackerCSRT_create,
                    'kcf': cv2.TrackerKCF_create,
                  'boosting':cv2.TrackerBoosting_create,
                  'mil':cv2.TrackerBoosting_create}


# csrt = accurate but slow
#kcf = fast and accurate....

trackers = cv2.MultiTracker_create()

In [16]:
if args['video_name'] == '':
    print('[INFO] starting video streaming')   
    vs = VideoStream(src=0).start()
    time.sleep(1.0)
    
else:                                                   #this will start webcam to capture the video
    vs = cv2.VideoCapture(args['video_name'])

    
while True:

    frame = vs.read()
    frame = frame[1] if args.get("video_name", False) else frame
# check to see if we have reached the end of the stream
    if frame is None:
        break
# resize the frame (so we can process it faster)
    frame = imutils.resize(frame, width=600)
# grab the updated bounding box coordinates (if any) for each
# object that is being tracked
    (success, boxes) = trackers.update(frame)
# loop over the bounding boxes and draw then on the frame
    for box in boxes:
        (x, y, w, h) = [int(v) for v in box]
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)           #(0,250,0 is denoted BGR)
        
# show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
# if the 's' key is selected, we are going to "select" a bounding
# box to track
    if key == ord("s"):
# select the bounding box of the object we want to track (make
# sure you press ENTER or SPACE after selecting the ROI)
        box = cv2.selectROI("Frame", frame, fromCenter=False,
            showCrosshair=False)
# create a new object tracker for the bounding box and add it
# to our multi-object tracker
        tracker = opencv_ob_track[args["tracker"]]()
        trackers.add(tracker, frame, box)
# if the `q` key was pressed, break from the loop
    elif key == ord("q"):
        break
# if we are using a webcam, release the pointer
if not args.get("video_name", False):
    vs.stop()
# otherwise, release the file pointer
else:
    vs.release()
# close all windows
cv2.destroyAllWindows()
        
        
        